In [1]:
import os
import openai
import json
 
openai.api_key = os.environ["OPENAI_API_KEY"]

def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0,
      max_tokens=1100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [2]:
import pandas as pd

calldata = pd.read_csv('chatgpt_calldata.csv')
calldata.head()

,doc,tags
0,상담 전화에서의 다양한 대화\n\n고객 보호조치 요청\n상담 녹음 안내\n진료 예약...,"고객 보호조치,상담 녹음,진료 예약"
1,친구와 저녁약속\n\n압구정 로데오\n사주카페\n금요일저녁,"사주카페,5만원,예약 필수"
2,상담 전화에서의 다양한 대화\n\n고객 보호조치 요청\n상담 녹음 안내\n진료 예약...,"고객 보호조치,상담 녹음,진료 예약"
3,친구와 저녁약속\n\n압구정 로데오\n사주카페\n금요일저녁,"사주카페,5만원,예약 필수"
4,상담 전화에서의 다양한 대화\n\n고객 보호조치 요청\n상담 녹음 안내\n진료 예약...,"고객 보호조치,상담 녹음,진료 예약"


In [7]:
calldata = calldata.drop_duplicates().reset_index(drop=True)
calldata.shape

(750, 2)

In [16]:
prompt = '''
Translate Input from Korean to English in Output.
Input:: 
1. 친구와 저녁약속\n\n압구정 로데오\n사주카페\n금요일저녁
2. 사주카페,5만원,예약 필수
Output:: 
1. Dinner with friends\n\nApgujeong Rodeo\nSaju Cafe\nFriday evening
2. Saju Cafe, 50,000 won, reservation required
Input:: 
1. {doc}
2. {tags}
Output:: 
'''

In [18]:
%%time
from datetime import datetime

eng_docs, eng_tags = [], []
start = datetime.now()
for i, (doc, tags) in enumerate(calldata.values):
    answer = llm(prompt.format(doc=doc, tags=tags), stop=['Input::'])
    eng_docs.append(answer.split('\n2. ')[0][2:].strip())
    eng_tags.append(answer.split('\n2. ')[1].strip())
    if i == 0 or i % 10 == 9:
        print(datetime.now() - start)
        print(i, eng_docs[i], eng_tags[i])
print(datetime.now() - start)
eng_calldata = pd.DataFrame({'eng_doc': eng_docs, 'eng_tags': eng_tags})

0:00:03.503505
0 Various conversations on the counseling phone

Request for customer protection measures
Counseling recording guidance
Reservation and inquiry for medical treatment Customer protection measures, counseling recording, medical reservation
0:00:28.997704
9 Business calls at the company

Report on the progress of the project
Adjust the meeting schedule
Division of work Project, meeting, division of work
0:00:54.501233
19 Talking about mother and family travel plans

Talking about travel plans with mother and family Travel, accommodation reservation
0:01:23.107704
29 Request for customer protection measures and inquiries about medical appointments

Request for customer protection measures
Guidance on recording consultation
Medical appointment and inquiry Customer protection measures, medical appointment
0:01:53.120144
39 English Conversation Class

Greetings
Daily Conversation
Movie Conversation
Workplace Conversation English, Conversation, Daily Conversation
0:02:29.979891


In [19]:
total_calldata = pd.concat([calldata, eng_calldata], 1)
total_calldata.head()

<ipython-input-19-6fffdb9888e8>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  total_calldata = pd.concat([calldata, eng_calldata], 1)


,doc,tags,eng_doc,eng_tags
0,상담 전화에서의 다양한 대화\n\n고객 보호조치 요청\n상담 녹음 안내\n진료 예약...,"고객 보호조치,상담 녹음,진료 예약",Various conversations on the counseling phone\...,"Customer protection measures, counseling recor..."
1,친구와 저녁약속\n\n압구정 로데오\n사주카페\n금요일저녁,"사주카페,5만원,예약 필수",Dinner with friends\n\nApgujeong Rodeo\nSaju C...,"Saju Cafe, 50,000 won, reservation required"
2,어머니와 의료 상담\n\n보호조치 요청\n상담 녹음 안내\n진료 예약 및 문의,"보호조치,상담녹음,진료예약",Medical consultation with mother\n\nRequest fo...,"Protective measures, counseling recording, med..."
3,친구와 여행 계획\n\n여름 휴가 계획\n해외 여행지 추천\n숙소 예약 방법,"여름휴가,해외여행,숙소예약",Travel plans with friends\n\nSummer vacation p...,"Summer vacation, overseas travel, accommodatio..."
4,상사와 업무 상담\n\n프로젝트 진행 상황 보고\n문제점 해결 방안 논의,"프로젝트진행,문제해결",Business consultation with the boss\n\nReport ...,"Project progress, problem solving"


In [20]:
total_calldata.shape

(750, 4)

In [21]:
total_calldata.to_csv('chatgpt_calldata_eng.csv', index=False)